In [ ]:
from llama_index.core import Settings
from llama_index.llms.groq import Groq

In [ ]:
import os
llm = Groq(model="llama-3.3-70b-versatile",
           api_key=os.environ["GROQ_API_KEY"],)

### 🧱 Criar uma função de calculo de imposto de renda, para se transformar em uma 🛠️Tool (🛠️ferramenta)

In [ ]:
def calcular_imposto_renda(rendimento: float) -> str:
    """
    Calcula o imposto de renda com base no rendimento anual.
    Args:
        rendimento (float): Rendimento anualdo individuo.
    Returns:
        str: Valor do imposto de renda a ser pago.
    """
    if rendimento <= 2000:
        return "Você está isento de pagar imposto de renda."
    elif 2000 < rendimento <= 5000:
        imposto = (rendimento - 2000) * 0.10
        return f"Você deve pagar R$ {imposto:.2f} baseado em um rendimento de R$ {rendimento:.2f}."
    elif 5000 < rendimento <= 10000:
        imposto = (rendimento - 5000) * 0.15 + (3000 * 0.10)
        return f"Você deve pagar R$ {imposto:.2f} baseado em um rendimento de R$ {rendimento:.2f}."
    else:
        imposto = (rendimento - 10000) * 0.20 + (5000 * 0.15) + (3000 * 0.10)
        return f"Você deve pagar R$ {imposto:.2f} baseado em um rendimento de R$ {rendimento:.2f}."

## 🔄 Convertendo 🧰 função em 🔧ferramenta

In [ ]:
from llama_index.core.tools import FunctionTool

#### 🧪 Definindo Função

In [ ]:
ferramenta_imposto_renda = FunctionTool.from_defaults(
    fn=calcular_imposto_renda,
    name="Calcular imposto de renda",
    description=(
        "Calcula o imposto de renda com base no rendimento anual do individuo."
        "Argumento: rendimento (float)."
        "Retorna o valor do imposto devido de acordo com as faixas de endimento."
    )
)


# ⚠️ Usando o Agente

In [ ]:
from llama_index.core.agent import FunctionCallingAgentWorker

In [ ]:
agent_worker_imposto = FunctionCallingAgentWorker.from_tools(
    tools=[ferramenta_imposto_renda],
    verbose=True,
    allow_parallel_tool_calls=True,
    llm=llm
)


#### 📦 Executar Agente 

In [ ]:
from llama_index.core.agent import AgentRunner

#### 🏷️ Importar Agent runner

In [ ]:
agent_imposto = AgentRunner(agent_worker_imposto)

In [ ]:
response = agent_imposto.chat("""
    QUal o imposto a ser pago por um rendimento anual de R$ 5000?
    """
)

#### 📂 Importar ARXIV

In [ ]:
import arxiv

def consulta_artigos(titulo: str) -> str:
    """
    Consulta os artigos na base de dados ArXiv e retorna resultados formatados.
    """
    busca = arxiv.Search(
        query=titulo,
        max_results=5,
        sort_by=arxiv.SortCriterion.Relevance
    )
    
    resultados = [
    f"Título: {artigo.title}\n"
    f"Categoria: {artigo.primary_category}\n"
    f"Link: {artigo.entry_id}\n"
    for artigo in busca.results()
    ]
    
    return "\n\n".join(resultados)

#### 🛠️ Transformar em ferramenta

In [ ]:
consulta_artigos_tool = FunctionTool.from_defaults(
    fn=consulta_artigos,
    name="Consulta artigos ArXiv",
    description=(
        "Consulta artigos na base de dados ArXiv com base no título fornecido."
        "Argumento: titulo (str)."
        "Retorna uma lista de artigos relevantes com título, categoria e link."
    )
)

#### 🧩 Agente Worker

> - usar o
```python
llm=llm
```
> - para chamar o llm do codigo, pois ele usa como padrão o OpenAI

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [ferramenta_imposto_renda, consulta_artigos_tool],
    verbose=True,
    allow_parallel_tool_calls=False,
    llm=llm
)

#### ⚙️ Executar o agent_runner

In [ ]:
agent = AgentRunner(agent_worker)
response = agent.chat("""
 Me retorne artigos sobre LangChain na educação
    """
)